<a href="https://colab.research.google.com/github/Rino-AR/Big-Data/blob/main/BigData_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Pengenalan Spark DataFrames
Spark DataFrame menyediakan struktur data yang optimal dengan operasi yang dioptimalkan untuk pemrosesan data besar, yang sangat mirip dengan DataFrame di Pandas atau di RDBMS.

- **Tugas 1**: Buat DataFrame sederhana di Spark dan eksplorasi beberapa fungsi dasar yang tersedia.

In [ ]:
# Contoh membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HandsOnPertemuan3').getOrCreate()

data = [('Pepe', 'Sales', 3000),
        ('Loughshinny', 'Sales', 4600),
        ('Eblana', 'Necromancer', 4100),
        ('Blemishine', 'Finance', 3000),
        ('Cellinia', 'Finance', 5000)]
columns = ['EmployeeName', 'Department', 'Salary']

df = spark.createDataFrame(data, schema=columns)
df.show()

+------------+-----------+------+
|EmployeeName| Department|Salary|
+------------+-----------+------+
|        Pepe|      Sales|  3000|
| Loughshinny|      Sales|  4600|
|      Eblana|Necromancer|  4100|
|  Blemishine|    Finance|  3000|
|    Cellinia|    Finance|  5000|
+------------+-----------+------+



In [ ]:
df.printSchema()

root
 |-- EmployeeName: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Salary: long (nullable = true)



In [ ]:
df.describe()

DataFrame[summary: string, EmployeeName: string, Department: string, Salary: string]

In [ ]:
df.describe().show()

+-------+------------+----------+-----------------+
|summary|EmployeeName|Department|           Salary|
+-------+------------+----------+-----------------+
|  count|           5|         5|                5|
|   mean|        NULL|      NULL|           3940.0|
| stddev|        NULL|      NULL|915.4233993076647|
|    min|  Blemishine|   Finance|             3000|
|    max|        Pepe|     Sales|             5000|
+-------+------------+----------+-----------------+



### 2. Transformasi Dasar dengan DataFrames
Pemrosesan data meliputi transformasi seperti filtering, selections, dan aggregations. Spark menyediakan cara efisien untuk melaksanakan operasi ini.

- **Tugas 2**: Gunakan operasi filter, select, groupBy untuk mengekstrak informasi dari data, serta lakukan agregasi data untuk mendapatkan insight tentang dataset menggunakan perintah seperti mean, max, sum.

In [ ]:
df.select('EmployeeName', 'Salary').show()

+------------+------+
|EmployeeName|Salary|
+------------+------+
|        Pepe|  3000|
| Loughshinny|  4600|
|      Eblana|  4100|
|  Blemishine|  3000|
|    Cellinia|  5000|
+------------+------+



In [ ]:
df.filter(df['Salary'] > 3000).show()

+------------+-----------+------+
|EmployeeName| Department|Salary|
+------------+-----------+------+
| Loughshinny|      Sales|  4600|
|      Eblana|Necromancer|  4100|
|    Cellinia|    Finance|  5000|
+------------+-----------+------+



In [ ]:
df.groupBy('Department').avg('Salary').show()

+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|      Sales|     3800.0|
|    Finance|     4000.0|
|Necromancer|     4100.0|
+-----------+-----------+



In [ ]:
from pyspark.sql.functions import count, mean, max, sum, min

insight_dataset = df.groupBy("Department") \
  .agg(
      count("EmployeeName").alias("Jumlah_Karyawan"),
      mean("Salary").alias("Rata_Rata_Gaji"),
      max("Salary").alias("Gaji_max"),
      min("Salary").alias("Gaji_min"),
      sum("Salary").alias("Total_Gaji")
  )

insight_dataset.show()

+-----------+---------------+--------------+--------+--------+----------+
| Department|Jumlah_Karyawan|Rata_Rata_Gaji|Gaji_max|Gaji_min|Total_Gaji|
+-----------+---------------+--------------+--------+--------+----------+
|      Sales|              2|        3800.0|    4600|    3000|      7600|
|    Finance|              2|        4000.0|    5000|    3000|      8000|
|Necromancer|              1|        4100.0|    4100|    4100|      4100|
+-----------+---------------+--------------+--------+--------+----------+



### 3. Bekerja dengan Tipe Data Kompleks
Spark mendukung tipe data yang kompleks seperti maps, arrays, dan structs yang memungkinkan operasi yang lebih kompleks pada dataset yang kompleks.

- **Tugas 3**: Eksplorasi bagaimana mengolah tipe data kompleks dalam Spark DataFrames.

In [ ]:
#Tipe 1
SalaryBonus = df.withColumn('SalaryBonus', df['Salary'] * 0.1)
SalaryBonus.show()

SalaryBonus.withColumn('TotalCompensation', SalaryBonus['Salary'] + SalaryBonus['SalaryBonus']).show()

+------------+-----------+------+-----------+
|EmployeeName| Department|Salary|SalaryBonus|
+------------+-----------+------+-----------+
|        Pepe|      Sales|  3000|      300.0|
| Loughshinny|      Sales|  4600|      460.0|
|      Eblana|Necromancer|  4100|      410.0|
|  Blemishine|    Finance|  3000|      300.0|
|    Cellinia|    Finance|  5000|      500.0|
+------------+-----------+------+-----------+

+------------+-----------+------+-----------+-----------------+
|EmployeeName| Department|Salary|SalaryBonus|TotalCompensation|
+------------+-----------+------+-----------+-----------------+
|        Pepe|      Sales|  3000|      300.0|           3300.0|
| Loughshinny|      Sales|  4600|      460.0|           5060.0|
|      Eblana|Necromancer|  4100|      410.0|           4510.0|
|  Blemishine|    Finance|  3000|      300.0|           3300.0|
|    Cellinia|    Finance|  5000|      500.0|           5500.0|
+------------+-----------+------+-----------+-----------------+



In [ ]:
#Tipe 2
df.withColumn('SalaryBonus', df['Salary'] * 0.1) \
  .withColumn('TotalCompensation', df['Salary'] + (df['Salary'] * 0.1)) \
  .show()

+------------+-----------+------+-----------+-----------------+
|EmployeeName| Department|Salary|SalaryBonus|TotalCompensation|
+------------+-----------+------+-----------+-----------------+
|        Pepe|      Sales|  3000|      300.0|           3300.0|
| Loughshinny|      Sales|  4600|      460.0|           5060.0|
|      Eblana|Necromancer|  4100|      410.0|           4510.0|
|  Blemishine|    Finance|  3000|      300.0|           3300.0|
|    Cellinia|    Finance|  5000|      500.0|           5500.0|
+------------+-----------+------+-----------+-----------------+



### 4. Operasi Data Lanjutan
Menggunakan Spark untuk operasi lanjutan seperti window functions, user-defined functions (UDFs), dan mengoptimalkan query.

- **Tugas 4**: Implementasikan window function untuk menghitung running totals atau rangkings.

In [ ]:
# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.partitionBy('Department').orderBy('Salary')
df.withColumn('Rank', F.rank().over(windowSpec)).show()

+------------+-----------+------+----+
|EmployeeName| Department|Salary|Rank|
+------------+-----------+------+----+
|  Blemishine|    Finance|  3000|   1|
|    Cellinia|    Finance|  5000|   2|
|      Eblana|Necromancer|  4100|   1|
|        Pepe|      Sales|  3000|   1|
| Loughshinny|      Sales|  4600|   2|
+------------+-----------+------+----+



### 5. Kesimpulan dan Eksplorasi Lebih Lanjut
Review apa yang telah dipelajari tentang pemrosesan data menggunakan Spark dan eksplorasi teknik lebih lanjut untuk mengoptimalkan pemrosesan data Anda.
<br>**Tugas 5**:
- Unduh dataset besar dari [Kaggle](https://www.kaggle.com/) atau sumber lainnya.
- Input data csv yang telah di download, kemudian load dan simpan data ke dalam pyspark.
- Setelah data berhasil di load menggunakan pyspark, lakukan manipulasi data untuk memperoleh informasi yang dibutuhkan

In [5]:
!pip install opendatasets

In [6]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/marcoalandadinanda/arknights-operators-dataset-global-server-may-2023")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Rino AR
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/marcoalandadinanda/arknights-operators-dataset-global-server-may-2023


100%|██████████| 48.9k/48.9k [00:00<00:00, 65.1MB/s]

In [7]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("operator_dataset").getOrCreate()

# Load dataset
df = spark.read.csv(
    "/content/arknights-operators-dataset-global-server-may-2023/operators_dataset.csv",
    header=True,
    inferSchema=True,
    sep=";" # delimiter ";"
)

# schema & contoh isi
df.printSchema()
df.show(5, truncate=False)

root
 |-- file_no: string (nullable = true)
 |-- name: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- class: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- position: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- faction: string (nullable = true)
 |-- infected_status: string (nullable = true)
 |-- infected_diagnosis: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- place_of_birth: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- race: string (nullable = true)
 |-- height_cm: double (nullable = true)
 |-- endurance: string (nullable = true)
 |-- strength: string (nullable = true)
 |-- arts_adaptability: string (nullable = true)
 |-- mobility: string (nullable = true)
 |-- tactical_acumen: string (nullable = true)
 |-- combat_skill: string (nullable = true)
 |-- experience_year: string (nullable = true)
 |-- base_hp: double (nullable = true)
 |-- elite_1_hp: double (nullable = true)
 

In [ ]:
df.describe()

DataFrame[summary: string, file_no: string, name: string, stars: string, class: string, branch: string, position: string, tags: string, faction: string, infected_status: string, infected_diagnosis: string, gender: string, place_of_birth: string, date_of_birth: string, race: string, height_cm: string, endurance: string, strength: string, arts_adaptability: string, mobility: string, tactical_acumen: string, combat_skill: string, experience_year: string, base_hp: string, elite_1_hp: string, elite_2_hp: string, max_hp: string, trust_hp: string, base_atk: string, elite_1_atk: string, elite_2_atk: string, max_atk: string, trust_atk: string, base_def: string, elite_1_def: string, elite_2_def: string, max_def: string, trust_def: string, base_res: string, elite_1_res: string, elite_2_res: string, max_res: string, base_redeploy_s: string, elite_1_redeploy_s: string, elite_2_redeploy_s: string, max_redeploy_s: string, base_dp_cost: string, elite_1_dp_cost: string, elite_2_dp_cost: string, max_dp_

In [ ]:
df.describe().show(5, truncate=False)

+-------+-------+-----------+------+--------+---------+--------+----------------------+---------------+---------------+------------------+----------+--------------+-------------+-----+------------------+---------+---------+-----------------+------------------+---------------+------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------------------------

In [ ]:
df.select('name', 'class','branch', 'faction', 'gender').show(5, truncate=False)

+--------------+----------+-----------+----------------+------+
|name          |class     |branch     |faction         |gender|
+--------------+----------+-----------+----------------+------+
|Castle-3      |Guard     |Dreadnought|Rhodes Island   |Male  |
|Justice Knight|Sniper    |Marksman   |Pinus Sylvestris|Female|
|Lancet-2      |Medic     |Medic      |Rhodes Island   |Female|
|THRM-EX       |Specialist|Executor   |Rhodes Island   |Male  |
|12F           |Caster    |Splash     |Rhodes Island   |Male  |
+--------------+----------+-----------+----------------+------+
only showing top 5 rows



In [ ]:
df.filter(df["stars"] == '6-star').show(10, truncate=False)


+-------+----------+------+----------+------------+--------+---------------------+-----------------------------+---------------+-------------------+------+--------------+-------------+--------+---------+-----------+---------+-----------------+---------+---------------+------------+---------------+-------+----------+----------+------+--------+--------+-----------+-----------+-------+---------+--------+-----------+-----------+-------+---------+--------+-----------+-----------+-------+---------------+------------------+------------------+--------------+------------+---------------+---------------+-----------+----------+-------------+-------------+---------+---------------+------------------+------------------+--------------+-----------------------------------+-------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------+---------

In [ ]:
df.groupBy("faction").count().show(truncate=False)

+----------------------------------------+-----+
|faction                                 |count|
+----------------------------------------+-----+
|Team Rainbow                            |4    |
|Kazimierz                               |6    |
|Followers                               |3    |
|Rhodes Island Operations Reserve Team A4|5    |
|Siracusa                                |6    |
|Yan                                     |5    |
|Blacksteel                              |4    |
|Sami                                    |2    |
|Glasgow                                 |2    |
|Bolívar                                 |3    |
|Karlan Commercial                       |8    |
|Victoria                                |20   |
|Higashi                                 |9    |
|Aegir                                   |2    |
|Lee's Detective Agency                  |4    |
|Iberia                                  |7    |
|Sargon                                  |11   |
|Rhodes Island Opera

In [9]:
from pyspark.sql.functions import col

jadi_int = ["base_hp", "base_atk", "base_def"]

for c in jadi_int:
    df = df.withColumn(c, col(c).cast("int"))

df.select(jadi_int).describe().show()


+-------+-----------------+-----------------+------------------+
|summary|          base_hp|         base_atk|          base_def|
+-------+-----------------+-----------------+------------------+
|  count|              235|              235|               235|
|   mean|904.5361702127659|258.1914893617021|117.17872340425532|
| stddev|302.6160330017471|92.71485518353542| 68.21423309437655|
|    min|              261|               42|                16|
|    max|             1882|              548|               354|
+-------+-----------------+-----------------+------------------+

